# **Setup Part**

## Install Libraries

In [ ]:
!pip install parsivar
!pip install emoji
!pip install -U tensorflow
!pip install fastapi kaleido python-multipart uvicorn
!pip install dadmatools
!pip install -U accelerate
!pip install -U transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8


## Import Necessary Libraries

In [ ]:
# fix the problem with numpy version's attributes and warnings
import numpy as np

def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator

np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from transformers import AutoTokenizer, get_linear_schedule_with_warmup, AdamW, get_scheduler,\
 AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,\
 pipeline
from datasets import DatasetDict, Dataset, load_dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm.notebook import tqdm
import pandas as pd
import os
from google.colab import drive

# Preprocessing Tools

In [ ]:
from parsivar import Normalizer
import regex as re

In [ ]:
def clean_persian_text(text):
    # Step 0: Use Parsivar to normalize data
    normalizer = Normalizer()
    text = normalizer.normalize(text)

    text = text.replace('٥', '5')

    # Step 1: Remove any English character from the text
    text = re.sub(r'[a-zA-Z]', '', text)

    # Step 2: Remove letters repeated more than twice
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Step 3: Remove any Arabic diacritics from the text
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)

    # Step 4: Remove the hashtag sign (“#”) from the text while keeping the information included in the hashtags
    text = re.sub(r'#', ' ', text)
    text = text.replace('_', ' ')

    # Step 5: Remove Persian and Arabic numeric characters from the text
    text = re.sub(r'\d+', '', text)

    # Step 6: Last normalization
    text = normalizer.normalize(text)

    return text

# **Evaluation Part**

## Load the Model

In [ ]:
# Setting checkpoint
checkpoint = 'xlm-roberta-large'

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

label_dict_reverse = {0: 'SAD', 1: 'HAPPY', 2: 'SURPRISE', 3: 'HATE', 4: 'FEAR', 5: 'ANGRY', 6: 'OTHER'}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/DL_model_v0.3")

## ArmanEmo Test Dataset

In [ ]:
progress = tqdm(total=2)

!wget https://raw.githubusercontent.com/Arman-Rayan-Sharif/arman-text-emotion/main/dataset/train.tsv -q
progress.set_description(f"Downloading Files")
progress.update(1)

!wget https://raw.githubusercontent.com/Arman-Rayan-Sharif/arman-text-emotion/main/dataset/test.tsv -q
progress.set_description(f"Downloading Files")
progress.update(1)

progress.close()

  0%|          | 0/2 [00:00<?, ?it/s]

## Read Evaluation Data

In [ ]:
file_path = 'test.tsv'

In [ ]:
header = ['text', 'label']
train_data = pd.read_csv('train.tsv', delimiter='\t', names=header)
test_data = pd.read_csv(file_path, delimiter='\t', names=header)

test_data.head()

,text,label
0,اين شايد اولين عزاى عمومى واقعى است كه ياد دار...,SAD
1,دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...,HAPPY
2,کدوم شعبه پول نداده، بگو الان برات آمار دقیق ...,OTHER
3,امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...,HAPPY
4,امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...,SAD


## Preprocess the Input Data

In [ ]:
train_data['text'] = train_data['text'].apply(clean_persian_text)
test_data['text'] = test_data['text'].apply(clean_persian_text)

test_data.head(100)

,text,label
0,این شاید اولین عزای عمومی واقعی است که یاد دار...,SAD
1,دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...,HAPPY
2,کدوم شعبه پول نداده ، بگو الان برات آمار دقیق ...,OTHER
3,امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...,HAPPY
4,امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...,SAD
...,...,...
95,در بازرسی سازمان تعزیرات ، یک شرکت لبنی معروف ...,SURPRISE
96,خوبه از بخاری لذت ببر چون چند وقت دیگه همینم ا...,ANGRY
97,یه وقتایی به یه سری محبت می‌کنی ، چون دوستشون‌...,SAD
98,کابل : دست کم دختر در یک مسابقه بایسکل سواری ا...,OTHER


## Replace Labels with Ids

In [ ]:
# Define your dictionary for replacement
label_dict = {'SAD': 0, 'HAPPY': 1, 'SURPRISE': 2, 'HATE': 3, 'FEAR': 4, 'ANGRY': 5, 'OTHER': 6}
label_dict_reverse = {0: 'SAD', 1: 'HAPPY', 2: 'SURPRISE', 3: 'HATE', 4: 'FEAR', 5: 'ANGRY', 6: 'OTHER'}

# Use the replace function
train_data['label'].replace(label_dict, inplace=True)
test_data['label'].replace(label_dict, inplace=True)

test_data.head()

,text,label
0,این شاید اولین عزای عمومی واقعی است که یاد دار...,0
1,دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...,1
2,کدوم شعبه پول نداده ، بگو الان برات آمار دقیق ...,6
3,امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...,1
4,امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...,0


In [ ]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6125
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1151
    })
})

## Tokenize the Input Data

In [ ]:
max_length = 128

def tokenize_and_split(examples):
    result = tokenizer(
        examples["text"],
        truncation=True,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        return_tensors='pt'
    )

    return result

dataset_dict = dataset_dict.map(tokenize_and_split, batched=True)

Map:   0%|          | 0/6125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

In [ ]:
final_dataset_dict = dataset_dict["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
final_dataset_dict["validation"] = final_dataset_dict.pop("test")
# Add the "test" set to our `DatasetDict`
final_dataset_dict["test"] = dataset_dict["test"]

## Evaluate the Input

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def get_trainer(model, tokenizer, tokenized_data):
    # Define where to store and evaluation strategy (step or epoch)
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=5,
        learning_rate=9e-6,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy='epoch',
        logging_dir='./logs',
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    return trainer

In [ ]:
def evaluate(model, tokenizer, tokenized_data):
    trainer = get_trainer(model, tokenizer, tokenized_data)
    test_data = tokenized_data['test']
    evaluation_result = trainer.predict(test_data)

    predictions = []
    outputs = evaluation_result[0]
    for row in outputs:
        predictions.append(label_dict_reverse[np.argmax(row).item()])

    results = evaluation_result[2]
    tests = evaluation_result[1]
    new_test_dataset = test_data.map(lambda x: {"label": label_dict_reverse[x["label"]]})
    texts = new_test_dataset['text']
    labels = new_test_dataset['label']

    csv_data = []
    for text, label, predicted in zip(texts, labels, predictions):
        row = [text, label, predicted]
        csv_data.append(row)

    columns = ["text", "label", "predicted"]
    df = pd.DataFrame(csv_data, columns=columns)
    df.to_csv('predictions.csv', index=False)

    return df, results

In [ ]:
df, results = evaluate(model, tokenizer, final_dataset_dict)

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

## Metrics for the Input Test Data

In [ ]:
print(results)

{'test_loss': 0.7461363077163696, 'test_accuracy': 0.7523892267593397, 'test_f1': 0.7526402555518705, 'test_precision': 0.7621035780970186, 'test_recall': 0.7523892267593397, 'test_runtime': 25.1208, 'test_samples_per_second': 45.819, 'test_steps_per_second': 2.866}
